# xLH-ai-oven
Eine exemplarische Umsetzung für die Integration von LLM-Modellen an Physikalische Umgebungen (IoT)

In [ ]:
import os
import pathlib
import sys
__cwd__ = str(pathlib.Path(os.getcwd())).replace('\\', '/')
sys.path.append(__cwd__)
print(__cwd__)
#%load_ext autoreload
#%autoreload 2

In [ ]:
from dataclasses import dataclass
from pydantic_ai import Tool
from rich import print
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from tools import set_oven_temperature
from llm_model import get_model, LlmModel
from object_to_file import base_model_to_file
from config import config
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# OpenAi API Key (Credentials hinterlegt in der Datei ..\xLH-mims-data\config\xlh_mims_python.env)
print(config.openai_api_key[:20])
# falls keine Datei im lokalen Speicher vorhanden ist, wir die Datei env.py

In [ ]:
@dataclass
class Deps:
    name: str

In [ ]:
class Ingredient(BaseModel):
    name: str = Field(description='Name der Zutat')
    quantity: str = Field(description='Menge')
    unit: str = Field(description='Einheit')

In [ ]:
class Recipe(BaseModel):
    title: str = Field(description='Name der Pizza')
    ingredients: list[Ingredient] = Field(description='Zutaten für die Menuezubereitung')
    preparation_steps: list[str] = Field(description='Zubereitungsschritte')

In [ ]:
model = get_model(llm_model=LlmModel.OPENAI_GPT_5_2)  # ToDo: Integration OLLAMA...
agent= Agent(
    model=model,
    system_prompt=('Du bist ein Pizzabäcker welcher Rezepte für kreative Pizzas kreirt. '
                   'Nutze das Tool set_oven_temperature für die Einstellung des Backofens.'),
    deps_type=Deps,
    tools=[
        Tool(set_oven_temperature, takes_ctx=True),
    ],
    retries=3,
    output_type=Recipe,
)
deps = Deps(name='-')

In [ ]:
response = agent.run_sync(user_prompt='Ich habe im Kühlschrank Lachs, Salami und Tomaten '
                                 'Kreire mir eine Pizza. '
                                 'Da mein Backofen etwas alterschwach ist, '
                                 'kann die Temperatur nicht höher als 225 Grad eingestellt werden '
                                 'und die Backzeit darf 30 Minuten nicht überschreiten.')

In [ ]:
result: Recipe = response.output
base_model_to_file(result)
# siehe recipe.json
print(f'Antwort: {result.model_dump_json(indent=4)[:100]}')

In [ ]:
usage = response.usage()
print(f'Input Tokens: {usage.input_tokens} ({usage.input_tokens*1.75*1e-6:0.04f} $)')
print(f'Output Tokens: {usage.output_tokens} ({usage.output_tokens*14.0*1e-6:0.04f} $)')
print(f'Total Tokens: {usage.total_tokens} (total cost: {(usage.input_tokens*1.75*1e-6 + usage.input_tokens*1.75*1e-6):0.04f} $)')

In [ ]:
settings = agent.model.settings
print(f'Settings: {settings}')
print(f'temperature: {settings.get('temperature')}')
print(f'presence_penalty: {settings.get('presence_penalty')}')
print(f'frequency_penalty: {settings.get('frequency_penalty')}')

In [ ]:
# END